# Calculator Development Notebook

This notebook was used to develop some of the core calculator logic.  Because notebooks are better for interacting w/ Pandas data frames than python scripts.

It has since been rewritten and incorporated into `calculate_taxes.py`

In [ ]:
import pandas as pd
from dataclasses import dataclass
from datetime import datetime

pd.set_option('display.max_rows', None)

# Read CSV

In [ ]:
raw_transactions = pd.read_csv('coinbase_pro_transactions.csv')

In [ ]:
raw_transactions

# Summarize single orders into single transactions

In [ ]:
buy_sells = raw_transactions[
    (raw_transactions['order id'].isnull() == False) &
    (raw_transactions['type'] == 'match')
]

In [ ]:
buy_sells.info()

## order_id | date | type (buy or sell) | currency | quantity | usd_amount

In [ ]:
@dataclass
class Transaction:
    order_id: str
    date: datetime
    usd_amount: float
    order_type: str
    currency: str
    currency_quantity: float
    
def transaction_from_group(name, group):    
    usd_amount = group[group['amount/balance unit'] == 'USD'].amount.sum()
    return Transaction(
        order_id=name,
        date=group.time.min(), 
        usd_amount=usd_amount,
        order_type='buy' if usd_amount < 0 else 'sell',
        currency=group[group['amount/balance unit'] != 'USD']['amount/balance unit'].min(),
        currency_quantity = group[group['amount/balance unit'] != 'USD'].amount.sum()
    )

In [ ]:
order_id_groups = buy_sells.groupby('order id', sort=False)

transactions = pd.DataFrame(transaction_from_group(name, group) for name, group in order_id_groups)

# Process each transaction in order

In [ ]:
@dataclass
class Position:
    currency: str
    purchase_date: datetime
    currency_quantity: float
    cost_basis: float
    sell_date: datetime = None
    proceeds: float = 0
    closed: bool = False

Position.__annotations__
    

In [ ]:
positions = pd.DataFrame(columns=Position.__annotations__.keys())

for ix, tx in transactions.iterrows():
    if tx.order_type == 'buy':
        new_position = Position(
            currency=tx.currency,
            purchase_date=tx.date,
            cost_basis=-tx.usd_amount,
            currency_quantity=tx.currency_quantity
        )
        positions = pd.concat([positions, pd.DataFrame([new_position])], ignore_index=True)
    else:
        open_orders_for_currency = positions[(positions['currency'] == tx.currency) & (positions['closed'] == False)]
        unsold_quantity = tx.currency_quantity
        for ix, open_position in open_orders_for_currency.iterrows():
            if open_position.currency_quantity + unsold_quantity == 0:
                positions.loc[ix]['closed'] = True
                positions.loc[ix]['proceeds'] = tx.usd_amount
                positions.loc[ix]['sell_date'] = tx.date
                unsold_quantity = 0
            elif open_position.currency_quantity + unsold_quantity > 0:
                positions.loc[ix]['closed'] = True
                positions.loc[ix]['proceeds'] = tx.usd_amount
                positions.loc[ix]['sell_date'] = tx.date
                positions.loc[ix]['cost_basis'] = open_position.cost_basis * -unsold_quantity / open_position.currency_quantity
                positions.loc[ix]['currency_quantity'] = -unsold_quantity
                new_position = Position(
                    currency=open_position.currency,
                    purchase_date=open_position.purchase_date,
                    cost_basis=open_position.cost_basis - positions.loc[ix]['cost_basis'],
                    currency_quantity=open_position.currency_quantity + unsold_quantity
                )
                positions = pd.concat([positions, pd.DataFrame([new_position])], ignore_index=True).sort_values(by="purchase_date")
                unsold_quantity = 0
            elif open_position.currency_quantity + unsold_quantity < 0:
                positions.loc[ix]['closed'] = True
                positions.loc[ix]['proceeds'] = tx.usd_amount * open_position.currency_quantity / -tx.currency_quantity
                positions.loc[ix]['sell_date'] = tx.date
                unsold_quantity += open_position.currency_quantity
                
            if unsold_quantity == 0:
                break
                
    
positions

In [ ]:
total_cost_basis = positions[positions['closed'] == True]['cost_basis'].sum()
total_proceeds = positions[positions['closed'] == True]['proceeds'].sum()

print("TOTAL COST BASIS:", total_cost_basis)
print("TOTAL PROCEEDS:", total_proceeds)